## Gemini API

**A very important and responsible moment!**:

---

### ⚠️ **You should not use the public Gemini API for sensitive corporate data unless you're using enterprise-grade controls.**

---

### 🛡️ Detailed Explanation:

#### 🚫 **Public Gemini API (AI Studio, Free or Basic Tier):**

* Your data **may be stored temporarily** for quality improvement, debugging, or abuse prevention.
* **Google may retain** the data you send unless you use specific enterprise tools.
* It's **not compliant with strict data privacy or confidentiality needs** by default.

🗒️ Source: [Google AI Terms](https://ai.google.dev/terms) (check for terms like *data use*, *retention*, *logging*, and *improvement*).

---

### ✅ **Safe Options for Handling Corporate Data:**

#### 1. **Vertex AI (Enterprise Gemini)**

* Google’s enterprise platform with **data isolation, encryption, compliance (e.g., ISO, HIPAA, GDPR)**.
* You can **disable data logging**.
* Fine-grained access control, audit logs, and VPC support.
* Paid, but suitable for serious business use.

#### 2. **On-Premise or Open-Source Models**

* For full control and privacy, consider self-hosted options like:

  * **OpenAI Whisper (local fine-tuned model)**.
  * **Open-source LLMs + speech models (e.g., OpenWhisper, WhisperX, etc.)**.

---

### ✅ What You Can Do (Safe Prototyping Strategy):

| Data Type                              | Public Gemini API | Vertex AI | Whisper (Local) |
| -------------------------------------- | ----------------- | --------- | --------------- |
| Sensitive (names, VÖEN, internal info) | ❌ No              | ✅ Yes     | ✅ Yes           |
| PII / Financial Data                   | ❌ No              | ✅ Yes     | ✅ Yes           |
| Anonymous samples                      | ⚠️ Maybe          | ✅ Yes     | ✅ Yes           |

---

### 🔐 Recommendation:

If you're dealing with, for example, **call center recordings, VÖENs, financial and internal IDs**, the **best path is one of the following**:

* **Use Whisper locally** on fine-tuned data.
* Or upgrade to **Vertex AI** with data protection settings.
---

In [1]:
# pip install google-genai

In [ ]:
import os
import time
from dotenv import load_dotenv
from google import genai

# === ENV SETUP ===
load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")

client = genai.Client(api_key=API_KEY)

# === CONFIGURATION ===
AUDIO_DIR = r"C:\...\Audio-Data"
OUTPUT_DIR = os.path.join(AUDIO_DIR, "transcripts")
MODEL = "models/gemini-2.5-flash"
DELAY_BETWEEN_REQUESTS = 6  # seconds (for Free Tier: 10 RPM)

os.makedirs(OUTPUT_DIR, exist_ok=True)

# === TRANSCRIBE FUNCTION WITH RETRY & RATE LIMIT ===
def transcribe_audio(file_path):
    try:
        print(f"📤 Uploading {file_path}")
        uploaded = client.files.upload(file=file_path)

        print("⏳ Transcribing...")
        response = client.models.generate_content(
            model=MODEL,
            contents=[
                "Transcribe this call center audio with timestamps [00:00] and speakers 'Speaker:' and 'Operator:'.",
                uploaded
            ]
        )
        return response.text

    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")
        return None

# === MAIN LOOP ===
for filename in os.listdir(AUDIO_DIR):
    if not filename.lower().endswith(".wav"):
        continue

    audio_path = os.path.join(AUDIO_DIR, filename)
    output_path = os.path.join(OUTPUT_DIR, os.path.splitext(filename)[0] + ".txt")

    # Skip if already transcribed
    if os.path.exists(output_path):
        print(f"✅ Already done: {filename}")
        continue

    transcription = transcribe_audio(audio_path)

    if transcription:
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(transcription)
        print(f"✅ Saved: {output_path}")
    else:
        print(f"⚠️ Failed to transcribe: {filename}")

    # Respect rate limits
    print(f"🕒 Sleeping {DELAY_BETWEEN_REQUESTS}s to respect rate limit...")
    time.sleep(DELAY_BETWEEN_REQUESTS)